# API Management operation statistics

With this notebook API Management all defined operations are fetched and matched with their request counts fetched from Application Insights.

## Requirements
- API Management integration to Application Insights
- environment variables set to your values (see _environment variables_)
- Service Principal in AAD with (see _creating a Service Principal for managing ARM resources_)
- adal package installed into current jupyter kernel (see _Install the adap pip packagel in the current Jupyter kernel_)

### environment variables
These environment variables need to be set with correct values 

| name | purpose | source |
| --- | --- | --- |
| AzMgmtAppId | AppId in AAD to allow access to ARM resources in your subscription | see 'creating a Service Principal for managing ARM resources' |
| AzMgmtPassword | Password for AppId above | see 'creating a Service Principal for managing ARM resources' |
| AzMgmtTenant | Tentant Id of AAD used to manage Azure subscription  | Azure Portal; AAD used to manage Azure subscription |
| AzMgmtSubscription | Id of subscription containing Api Management and Application Insights | Azure Portal |
| AzApiMgmtResourceGroup | Resource Group containing Api Management | Azure Portal |
| AzApiMgmtServiceName | API Management Service Name | Azure Portal |
| AzAppInsightsAppId | Application Insights AppId | Azure Portal |
| AzAppInsightsApiKey | Application Insights ApiKey used for REST API access | Azure Portal |

This can be achieved with a PowerShell setting the environment variables for the current session and for the user persistently:
```
Write-Host "setting Azure Environment secrets"
$env:AzMgmtAppId = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtAppId",$env:AzMgmtAppId,"User")

$env:AzMgmtPassword = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtPassword",$env:AzMgmtPassword,"User")

$env:AzMgmtTenant = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtTenant",$env:AzMgmtTenant,"User")

$env:AzMgmtSubscription = "{your-value}"
[Environment]::SetEnvironmentVariable("AzMgmtSubscription",$env:AzMgmtSubscription,"User")

$env:AzApiMgmtResourceGroup = "{your-value}"
[Environment]::SetEnvironmentVariable("AzApiMgmtResourceGroup",$env:AzApiMgmtResourceGroup,"User")

$env:AzApiMgmtServiceName = "{your-value}"
[Environment]::SetEnvironmentVariable("AzApiMgmtServiceName",$env:AzApiMgmtServiceName,"User")

$env:AzAppInsightsAppId = "{your-value}"
[Environment]::SetEnvironmentVariable("AzAppInsightsAppId",$env:AzAppInsightsAppId,"User")

$env:AzAppInsightsApiKey = "{your-value}"
[Environment]::SetEnvironmentVariable("AzAppInsightsApiKey",$env:AzAppInsightsApiKey,"User")
```

### creating a Service Principal for managing ARM resources
To create a Service Principal with PowerShell login to your AAD/Azure subscription with AzureRM, switch to the target subscription (containing API Management and Application Insights resources) and execute these steps:

```
$sp = New-AzureRmADServicePrincipal -DisplayName myAzureManagement -Password "thepassword"
New-AzureRmRoleAssignment -RoleDefinitionName Contributor -ServicePrincipalName $sp.ApplicationId
```

Note ```ApplicationId``` and put it into the environment variables.

### Install the adap pip packagel in the current Jupyter kernel
To install adal package into your kernel, copy these code lines into a newly inserted cell and execute.

```python
import sys
!{sys.executable} -m pip install adal
``` 

In [1]:
# obtain authorization token to be used in API Management REST API
import adal
import os
import json

def getToken():
    AppId = os.environ.get('AzMgmtAppId')
    AppPassword = os.environ.get('AzMgmtPassword')
    tenantID = os.environ.get('AzMgmtTenant')
    authURL = "https://login.windows.net/" + tenantID
    resource = "https://management.azure.com/"

    context = adal.AuthenticationContext(
        authURL, validate_authority=tenantID, api_version=None)

    token = context.acquire_token_with_client_credentials(
        resource,
        AppId,
        AppPassword)

    return token

def getAuthorizationHeader():
    token = getToken()

    header = token['tokenType'] + ' ' + token['accessToken']

    return header


# ----------------------------------------------------------------------

authHeader = getAuthorizationHeader()

print(authHeader)


Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlRpb0d5d3dsaHZkRmJYWjgxM1dwUGF5OUFsVSIsImtpZCI6IlRpb0d5d3dsaHZkRmJYWjgxM1dwUGF5OUFsVSJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tLyIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzgyOTEzZDkwLTg3MTYtNDAyNS1hOGU4LTRmOGRmYTQyYjcxOS8iLCJpYXQiOjE1MzE2MzI2NjQsIm5iZiI6MTUzMTYzMjY2NCwiZXhwIjoxNTMxNjM2NTY0LCJhaW8iOiI0MkJnWU5qZm5TNFptY2w5UGY3YWJLYktybG5sQUE9PSIsImFwcGlkIjoiOGJlMWU2MjQtNmI3NS00ZTYzLTlmMzYtNWNjMzVjZWE0ZThmIiwiYXBwaWRhY3IiOiIxIiwiZV9leHAiOjI2MjgwMCwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvODI5MTNkOTAtODcxNi00MDI1LWE4ZTgtNGY4ZGZhNDJiNzE5LyIsIm9pZCI6Ijk3NjIyOWJmLTU4NDctNDZlZS1hODVjLWI1OTI5M2FlYzVjMyIsInN1YiI6Ijk3NjIyOWJmLTU4NDctNDZlZS1hODVjLWI1OTI5M2FlYzVjMyIsInRpZCI6IjgyOTEzZDkwLTg3MTYtNDAyNS1hOGU4LTRmOGRmYTQyYjcxOSIsInV0aSI6InpaUGZKdGdOdVVTSU9XbmFiQW9IQUEiLCJ2ZXIiOiIxLjAifQ.UAPDhV9uvGJ2NhRF7dVr8WWFdBU9jR30jbEb82fZoHL7vxHu9m6t_NbujYuZWzhDrsKS83YfCfAp4vcBlAXpSGh9vYk6t8Wz9EgzQpl9tvHmONxrc82V2A4eceV6bwRUfUKb1oBGWLmAsXYNCWNW9H2UAfq-c

In [2]:
# use API Management REST API to obtain APIs
import requests

resource = "https://management.azure.com"
resourceGroupName = os.environ.get('AzApiMgmtResourceGroup')
serviceName = os.environ.get('AzApiMgmtServiceName')
subscriptionId = os.environ.get('AzMgmtSubscription')
apiVersionSuffix = "?api-version=2017-03-01"
        
url = '{}/subscriptions/{}/resourceGroups/{}/providers/Microsoft.ApiManagement/service/{}/apis{}'.format(resource,subscriptionId,resourceGroupName,serviceName,apiVersionSuffix)
headers = {"Content-Type": "application/json","Authorization":authHeader}

response = requests.get(url,headers=headers)


In [3]:
print(response.json()['value'][:2])

[{'id': '/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-adh-core-inbound', 'type': 'Microsoft.ApiManagement/service/apis', 'name': 'imt-adh-core-inbound', 'properties': {'displayName': 'IMT Assembly Data Hub - Core - Inbound', 'apiRevision': '1', 'description': 'IMT Assembly Data Hub (ADH) Inbound', 'serviceUrl': 'https://esb-stage.zeiss.com/public/api/imt/adh/core/inbound', 'path': 'public/api/imt/adh/core/inbound', 'protocols': ['https'], 'authenticationSettings': None, 'subscriptionKeyParameterNames': None, 'isCurrent': True}}, {'id': '/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-adh-proboard-outbound', 'type': 'Microsoft.ApiManagement/service/apis', 'name': 'imt-adh-proboard-outbound', 'properties': {'displayName': 'IMT Assembly Data Hub - Core - Outbound', 'apiRevision': '1', 'descrip

In [4]:
# transform API result into dataframe
import pandas as pd
apis = pd.DataFrame.from_dict(response.json()['value'])
apis.head(n=2)

,id,name,properties,type
0,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,imt-adh-core-inbound,{'displayName': 'IMT Assembly Data Hub - Core ...,Microsoft.ApiManagement/service/apis
1,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,imt-adh-proboard-outbound,{'displayName': 'IMT Assembly Data Hub - Core ...,Microsoft.ApiManagement/service/apis


In [5]:
# pull path from properties into data frame
path = apis.properties.apply(lambda x : x["path"])
apis = apis.assign(path=path)
apis.head(n=2)


,id,name,properties,type,path
0,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,imt-adh-core-inbound,{'displayName': 'IMT Assembly Data Hub - Core ...,Microsoft.ApiManagement/service/apis,public/api/imt/adh/core/inbound
1,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,imt-adh-proboard-outbound,{'displayName': 'IMT Assembly Data Hub - Core ...,Microsoft.ApiManagement/service/apis,public/api/imt/adh/core/outbound


In [6]:
# create a data frame with all operations in all APIs
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import urllib

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# get operations for a single API
def getOperations(id):
    url = '{}{}/operations{}'.format(resource,urllib.parse.quote(id),apiVersionSuffix)
    print(url)
    headers = {"Content-Type": "application/json","Authorization":authHeader}
    response = requests_retry_session().get(url,headers=headers)
    print(response.status_code)
    return response.json()["value"]

# loop thru all APIs, fetch operations
for index, api in apis.iterrows():
    print(index,api.id)
    apiOperations = pd.DataFrame.from_dict(getOperations(api.id))
    apiOperations['apiPath'] = api.path
    apiOperations['operation'] = apiOperations.apply(lambda x:'%s %s' % (x.properties['method'],x.properties['urlTemplate']),axis=1)
    if index == 0:
        operations = apiOperations
    else:
        operations = operations.append(apiOperations,ignore_index=True)
    
operations.head()

0 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-adh-core-inbound
https://management.azure.com/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-adh-core-inbound/operations?api-version=2017-03-01
200
1 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-adh-proboard-outbound
https://management.azure.com/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-adh-proboard-outbound/operations?api-version=2017-03-01
200
2 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/imt-usapprize-inbound
https://management.azure.com/subscri

200
21 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/5ad723bd8934640216b53564
https://management.azure.com/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/5ad723bd8934640216b53564/operations?api-version=2017-03-01
200
22 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/5b0e40dd9650f0f0ddc8b2ff
https://management.azure.com/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/5b0e40dd9650f0f0ddc8b2ff/operations?api-version=2017-03-01
200
23 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/zeiss-fico–invoices–inbound
https://managemen

200
41 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/zeiss-internal-dcc-ibases
https://management.azure.com/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/zeiss-internal-dcc-ibases/operations?api-version=2017-03-01
200
42 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/39c75c1888754a1d96b3f1896ba2ffe4
https://management.azure.com/subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/39c75c1888754a1d96b3f1896ba2ffe4/operations?api-version=2017-03-01
200
43 /subscriptions/a5d101c8-7211-4151-9fb1-ab054271927b/resourceGroups/ZEISS.ESB.PROD/providers/Microsoft.ApiManagement/service/zeissesbprod/apis/sip-dcc-outbound
https://ma

,apiPath,id,name,operation,properties,type
0,public/api/imt/adh/core/inbound,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,5a9821194fb40e0d0003fabf,PUT /confirmation/update,"{'displayName': 'Confirmations', 'method': 'PU...",Microsoft.ApiManagement/service/apis/operations
1,public/api/imt/adh/core/inbound,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,5a9821194fb40e0d0003fac0,PUT /delivery/update,"{'displayName': 'Deliveries', 'method': 'PUT',...",Microsoft.ApiManagement/service/apis/operations
2,public/api/imt/adh/core/inbound,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,5a992aa9588c821e981f7bb9,POST /missingpart/create,"{'displayName': 'Missing Parts', 'method': 'PO...",Microsoft.ApiManagement/service/apis/operations
3,public/api/imt/adh/core/inbound,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,5a9821194fb40e0d0003fac1,PUT /planned-order/update,"{'displayName': 'Planned Orders', 'method': 'P...",Microsoft.ApiManagement/service/apis/operations
4,public/api/imt/adh/core/inbound,/subscriptions/a5d101c8-7211-4151-9fb1-ab05427...,5a9821194fb40e0d0003fabe,PUT /production-order/update,"{'displayName': 'Production Orders', 'method':...",Microsoft.ApiManagement/service/apis/operations


In [7]:
import requests
import urllib
import pandas as pd

appInsightsAppId = os.environ.get('AzAppInsightsAppId')
appInsightsApiKey = os.environ.get('AzAppInsightsApiKey')
appInsightsQuery = 'requests | where timestamp >= ago(7d) and tostring(customDimensions.ApimanagementServiceName) != "" and name !startswith "OPTIONS" and success == true | project apiOperation=tostring(customDimensions["Operation Name"]),itemCount | summarize calls=count() by  apiOperation'
appInsightsUrl = "https://api.applicationinsights.io/v1/apps/{}".format(appInsightsAppId)

url = '{}/query?query={}'.format(appInsightsUrl,urllib.parse.quote(appInsightsQuery))
headers = {"Content-Type": "application/json","x-api-key":appInsightsApiKey}

response = requests.get(url,headers=headers)

stats = pd.DataFrame.from_dict(response.json()["tables"][0]["rows"])
stats.columns = ['name','counter']
stats.head()

,name,counter
0,594b6171f4255025c1db4594,150034
1,5a79fa254fb40e170085e59e,13166
2,5ac8f1284fb40e01b8b062de,150182
3,5a9cea784fb40e0d0003ff3f,10161
4,58e5de271b9b1a10bcc11163,198286


In [30]:
operationResults = pd.merge(operations,stats,on='name',how='left')[['apiPath','operation','counter']]
operationResults['counter'].fillna(0,inplace=True)
operationResults.head()

,apiPath,operation,counter
0,public/api/imt/adh/core/inbound,PUT /confirmation/update,2549.0
1,public/api/imt/adh/core/inbound,PUT /delivery/update,8336.0
2,public/api/imt/adh/core/inbound,POST /missingpart/create,1.0
3,public/api/imt/adh/core/inbound,PUT /planned-order/update,4644.0
4,public/api/imt/adh/core/inbound,PUT /production-order/update,781.0


In [31]:
operationResults.sort_values(by='counter', ascending=False)

,apiPath,operation,counter
85,public/api/core/vis/input,POST /orderstatus,478980.0
140,public/api/licensing/entitlement,POST /EntitlementItems/findBySerialNumbers,259147.0
84,public/api/core/vis/input,POST /orderlog,198286.0
231,private/api/core/outbound,PUT /BusinessPartnerGeneral-Changed,167120.0
230,private/api/core/outbound,PUT /BusinessPartnerAddress-Changed,164252.0
213,public/api/internal/accessory,GET /mnf/imt/adh/check-production-order/{productionorderno}/plant/{plantno}/ordertype/{ordertype},150182.0
249,private/api/core/outbound,PUT /ProductionOrder-Changed,150034.0
285,public/api/logistics/tracking/eurolog/inbound,POST /delivery-tracking,130631.0
247,private/api/core/outbound,PUT /PlannedOrder-Changed,120478.0
158,public/api/core,GET /customer/{customerid},75091.0
